<a href="https://colab.research.google.com/github/sfansaria/CrimePredictionUsingLSTM-RandomForestClassifier-RandomForestRegressor-ARIMA/blob/main/Crime_rf_regressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import math

from IPython.display import HTML, display
import plotly.graph_objects as go



from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error



In [43]:

crime_df = pd.read_csv('/content/drive/My Drive/ml_datasets/Crime_Data_from_2020_to_Present.csv')

In [44]:
dict_zone = {}
for index, row in crime_df.iterrows():
    if( not (int(row['AREA']) in dict_zone)):
        dict_zone[row['AREA']] = row['AREA NAME']
dict_zone

{7: 'Wilshire',
 1: 'Central',
 3: 'Southwest',
 9: 'Van Nuys',
 6: 'Hollywood',
 18: 'Southeast',
 13: 'Newton',
 19: 'Mission',
 2: 'Rampart',
 4: 'Hollenbeck',
 10: 'West Valley',
 8: 'West LA',
 20: 'Olympic',
 21: 'Topanga',
 11: 'Northeast',
 12: '77th Street',
 14: 'Pacific',
 15: 'N Hollywood',
 5: 'Harbor',
 16: 'Foothill',
 17: 'Devonshire'}

In [45]:
zonaUnderExam_id = 14
zonaUnderExam_name = dict_zone[zonaUnderExam_id]
print("ID=",zonaUnderExam_id, "Name=",zonaUnderExam_name)

ID= 14 Name= Pacific


In [46]:
crime_df['month'] = pd.to_datetime(crime_df['DATE OCC']).dt.strftime('%m/%Y')
crime_df.head(5)

<ipython-input-46-2bf2fd79b2e6>:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,month
0,190326475,03/01/2020 12:00:00 AM,03/01/2020 12:00:00 AM,2130,7,Wilshire,784,1,510,VEHICLE - STOLEN,...,Adult Arrest,510.0,998.0,NaN,NaN,1900 S LONGWOOD AV,NaN,34.0375,-118.3506,03/2020
1,200106753,02/09/2020 12:00:00 AM,02/08/2020 12:00:00 AM,1800,1,Central,182,1,330,BURGLARY FROM VEHICLE,...,Invest Cont,330.0,998.0,NaN,NaN,1000 S FLOWER ST,NaN,34.0444,-118.2628,02/2020
2,200320258,11/11/2020 12:00:00 AM,11/04/2020 12:00:00 AM,1700,3,Southwest,356,1,480,BIKE - STOLEN,...,Invest Cont,480.0,NaN,NaN,NaN,1400 W 37TH ST,NaN,34.0210,-118.3002,11/2020
3,200907217,05/10/2023 12:00:00 AM,03/10/2020 12:00:00 AM,2037,9,Van Nuys,964,1,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),...,Invest Cont,343.0,NaN,NaN,NaN,14000 RIVERSIDE DR,NaN,34.1576,-118.4387,03/2020
4,220614831,08/18/2022 12:00:00 AM,08/17/2020 12:00:00 AM,1200,6,Hollywood,666,2,354,THEFT OF IDENTITY,...,Invest Cont,354.0,NaN,NaN,NaN,1900 TRANSIENT,NaN,34.0944,-118.3277,08/2020


In [47]:
Zone_Month = crime_df.groupby(["AREA","month"]).size().reset_index(name='Count')
Zone_Month.head(5)

,AREA,month,Count
0,1,01/2020,1211
1,1,01/2021,886
2,1,01/2022,1290
3,1,01/2023,1550
4,1,01/2024,1428


In [48]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i+look_back, 0])
    return np.array(dataX), np.array(dataY)

In [49]:
AREA = zonaUnderExam_id

dataset = Zone_Month[(Zone_Month['AREA']==AREA)]
drop_attributes = ['AREA','month']
dataset = dataset.drop(columns=drop_attributes)
#dataset = dataset.drop('AREA', 1)
#dataset = dataset.drop('month', 1)
print(dataset)
print(len(dataset))

     Count
650    944
651    982
652   1118
653   1122
654    959
655    919
656    997
657    959
658   1050
659    434
660    884
661   1046
662   1213
663   1128
664    877
665   1039
666   1162
667   1168
668    906
669   1166
670   1138
671   1106
672    949
673   1070
674   1017
675   1159
676   1065
677   1152
678   1097
679   1219
680   1073
681   1186
682   1106
683   1165
684   1019
685   1061
686    988
687   1192
688    996
689   1070
690   1159
691   1162
692    975
693   1056
694    990
695   1088
696    959
697   1094
698   1080
699   1073
50


In [50]:
r = np.arange(0,50,1).tolist()
dataset['month_count'] = r
dataset

,Count,month_count
650,944,0
651,982,1
652,1118,2
653,1122,3
654,959,4
655,919,5
656,997,6
657,959,7
658,1050,8
659,434,9


In [51]:
x = dataset['month_count'].values.reshape(-1,1)
y = dataset['Count'].values.reshape(-1,1)
x_train_rdf, x_test_rdf, y_train_rdf, y_test_rdf = train_test_split(x,y,test_size=0.30)


In [52]:
rd = RandomForestRegressor()
rd.fit(x_train_rdf, y_train_rdf)

<ipython-input-52-5b13105dc475>:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestRegressor()

In [53]:
y_pred_rdf = rd.predict(x_test_rdf)
y_test_rdf, y_pred_rdf

(array([[1162],
        [1019],
        [ 944],
        [1080],
        [1097],
        [ 975],
        [1073],
        [ 877],
        [1094],
        [ 919],
        [1128],
        [ 996],
        [1070],
        [1168],
        [1056]]),
 array([1035.6 , 1133.72, 1024.18, 1005.85, 1154.1 , 1151.44, 1005.85,
        1075.39, 1005.85,  981.65, 1149.38, 1148.74,  996.25,  977.69,
        1051.81]))

In [54]:
# Plot the data.
listGraph = []
listGraph.append(go.Scatter(x=list(range(0,35)),y=y_test_rdf.flatten(),name="Test set"))
listGraph.append(go.Scatter(x=list(range(0,35)),y=y_pred_rdf,name="Predict set"))
fig = go.Figure(data=listGraph)
fig.update_layout(width=1024,height=500,title="Prediction for "+dict_zone[AREA],xaxis=dict(title='Month',tickmode='linear'),yaxis=dict(title='Number of crimes'))

fig.show()

In [55]:
last_df = pd.DataFrame()
last_df['Test'] = y_test_rdf.flatten()
last_df['Predicted'] = y_pred_rdf
last_df

,Test,Predicted
0,1162,1035.60
1,1019,1133.72
2,944,1024.18
3,1080,1005.85
4,1097,1154.10
5,975,1151.44
6,1073,1005.85
7,877,1075.39
8,1094,1005.85
9,919,981.65
